In [0]:
%sql
select * from main_prod.earnings_analysis.fact_user_earnings_daily

In [0]:
%sql
select * from main_prod.ml_data.static_moving_worktype where paydate > '2025-07-01'

In [0]:
%sql
select * from main_prod.ml_features.static_moving_features where date(predtime)> '2025-07-01' order by predtime


In [0]:
# check the static moving features in the month of july

query = """
select userid, predtime from main_prod.ml_features.static_moving_features
"""
df = spark.sql(query)
display(df)


In [0]:
facts_df = spark.read.table("main_prod.earnings_analysis.fact_user_earnings_daily")
display(facts_df)

In [0]:
from pyspark.sql import functions as F
facts_df_v2 = facts_df.withColumn("predtime", F.expr("date_sub(to_date(prediction_day), 1)").cast("string"))
display(facts_df_v2)

In [0]:
query = """
select userid, predtime from main_prod.ml_features.static_moving_features where is_static is not null
"""
static_moving_df = spark.sql(query)
display(static_moving_df)

In [0]:
joined_df = facts_df_v2.join(static_moving_df, ["userid", "predtime"], "inner").drop("predtime")
# display(joined_df)

In [0]:
joined_df.write.mode('overwrite').saveAsTable('main_prod.datascience_scratchpad.fact_user_earnings_daily_static_moving_fil')

In [0]:
joined_df = spark.read.table('main_prod.datascience_scratchpad.fact_user_earnings_daily_static_moving_fil')
joined_df.count()

In [0]:
147100283

In [0]:
display(joined_df)

In [0]:
joined_df.where("prediction_day>='2025-07-01' and prediction_day<='2025-07-31'").count()

In [0]:
%sql
select count(*) from main_prod.earnings_analysis.fact_user_earnings_daily